In [1]:
import tensorflow as tf
import datetime, os
%automagic


Automagic is OFF, % prefix IS needed for line magics.


In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard


In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


In [4]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])


In [5]:
def train_model():

  model = create_model()
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(x=x_train, 
            y=y_train, 
            epochs=5, 
            validation_data=(x_test, y_test), 
            callbacks=[tensorboard_callback])

train_model()


Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.4957 - accuracy: 0.8226 - val_loss: 0.4382 - val_accuracy: 0.8419
Epoch 2/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.3810 - accuracy: 0.8610 - val_loss: 0.3788 - val_accuracy: 0.8639
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3461 - accuracy: 0.8730 - val_loss: 0.3563 - val_accuracy: 0.8701
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3247 - accuracy: 0.8798 - val_loss: 0.3644 - val_accuracy: 0.8670
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3129 - accuracy: 0.8835 - val_loss: 0.3606 - val_accuracy: 0.8712


In [6]:
%tensorboard --logdir logs


Reusing TensorBoard on port 6006 (pid 3652), started 8 days, 0:28:34 ago. (Use '!kill 3652' to kill it.)